# Crypto Writeups for GreyCTF 2023

I played with my team Social Engineering Experts, though we called ourselves The JoshL Fan Club for some reason. We came in eighth overall (and top in Singapore), having solved all challenges except `misc/beepboop2`.

There were nine crypto challenges, listed here in decreasing order of solves:
- Baby Crypto (50 points / 350 solves)
- EncryptService (50 points / 105 solves) 🩸
- The Vault (50 points / 100 solves)
- GreyCat Trial (50 points / 63 solves)
- OT (212 points / 41 solves)
- PLCG (328 points / 32 solves)
- Encrypt (421 points / 22 solves)
- Fancy (428 points / 21 solves) 🥈
- QRSA (454 points / 17 solves) 🩸

We will provide a brief solve description for all of them. But first we import some standard stuff.

In [1]:
from sage.all import *
from pwn import *
from Crypto.Util.number import *
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
from hashlib import sha256, shake_256

## Baby Crypto (50 points / 350 solves)

We are given `whuo{squi4h_s1fx3h_v0h_co_i4b4T}`. This is just a [Caesar cipher](https://en.wikipedia.org/wiki/Caesar_cipher), so we can [solve it using CyberChef](https://gchq.github.io/CyberChef/#recipe=ROT13(true,true,false,10)&input=d2h1b3tzcXVpNGhfczFmeDNoX3YwaF9jb19pNGI0VH0).

The flag is `grey{caes4r_c1ph3r_f0r_my_s4l4D}`.

## EncryptService (50 points / 105 solves) 🩸

We encrypt the all-zero plaintext, and one of the ciphertexts given will be our flag (xored with the encrypted flag).

In [2]:
with remote('34.124.157.94', 10590) as sh:
    sh.sendline(b'0' * 128)
    cts = [bytes.fromhex(sh.readline_contains(b'Cipher').decode().split()[-1]) for _ in range(256)]
    flag = bytes.fromhex(sh.readline_contains(b'Flag').decode().split()[-1])
for ct in cts:
    tmp = xor(flag, ct, cut='min')
    if tmp.startswith(b'grey{'):
        print(tmp)

[x] Opening connection to 34.124.157.94 on port 10590
[x] Opening connection to 34.124.157.94 on port 10590: Trying 34.124.157.94
[+] Opening connection to 34.124.157.94 on port 10590: Done
[*] Closed connection to 34.124.157.94 port 10590
b'grey{0h_m4n_57r34m_c1ph3r_n07_50_53cur3}'


## The Vault (50 points / 100 solves)

We submit $a=10^{129}+1$ and $b=1$. This just becomes $a \equiv 1 \pmod{10^{128}}$, so the condition is trivial.

In [3]:
with remote('34.124.157.94', 10591) as sh:
    sh.sendlines([str(10**129+1).encode(), b'1'])
    print(sh.readline_contains(b'grey{'))

[x] Opening connection to 34.124.157.94 on port 10591
[x] Opening connection to 34.124.157.94 on port 10591: Trying 34.124.157.94
[+] Opening connection to 34.124.157.94 on port 10591: Done
b'grey{th3_4n5w3R_T0_Th3_3x4M_4nD_3v3ry7H1N6_1s_42}'
[*] Closed connection to 34.124.157.94 port 10591


## GreyCat Trial (50 points / 63 solves)

According to this [page](http://primerecords.dk/aprecords.htm#ap24) tracking records for primes in arithmetic progression, the smallest known end for an AP-26 is:

```3486107472997423 + 1666981·23#·n (12783396861134173)```

We technically need an AP-27, but will have to work with this because the best known AP-27 is too big. We note that the success rate is approximately $1/e \simeq 37\%$, and can just keep retrying until it works.

In [4]:
assert (12783396861134173).bit_length() < 55
while True:
    try:
        with remote('34.124.157.94', 10592) as sh:
            sh.sendlines([b'3486107472997423', str(1666981*product(primes(23+1))).encode()])
            print(sh.readline_contains(b'grey{'))
            break
    except:
        pass

[x] Opening connection to 34.124.157.94 on port 10592
[x] Opening connection to 34.124.157.94 on port 10592: Trying 34.124.157.94
[+] Opening connection to 34.124.157.94 on port 10592: Done
b'grey{Gr33N-tA0_The0ReM_w1z4rd}'
[*] Closed connection to 34.124.157.94 port 10592


## OT (212 points / 41 solves)

If we have a prime $p$ with known factorisation of $p-1$, then we can just pick $N = 23(p-1)$. Let's construct such a $p$.

Note that it fails sometimes because one of the keys is larger than the order, in which case we keep retrying until it works.

In [5]:
for i in range(2000, 3000):
    p = i * 2**4080 + 1
    if is_prime(p):
        break
        
while True:
    with remote('34.124.157.94', 10521) as sh:
        sh.sendline(str(23*(p-1)).encode())
        c1 = int(sh.readline_contains(b' = ').split()[-1])
        c2 = int(sh.readline_contains(b' = ').split()[-1])
        enc_flag = bytes.fromhex(sh.readline_contains(b' = ').decode().split()[-1])

    k1, k2 = [pow(c, pow(65537, -1, int(euler_phi(N))), N) for c, N in [(c1, 23*(p-1)), (c2, 23*p)]]
    flag = xor(enc_flag, shake_256(long_to_bytes(k1 ^ k2)).digest(len(enc_flag)))
    if flag.startswith(b'grey{'):
        print(flag)
        break

[x] Opening connection to 34.124.157.94 on port 10521
[x] Opening connection to 34.124.157.94 on port 10521: Trying 34.124.157.94
[+] Opening connection to 34.124.157.94 on port 10521: Done
[*] Closed connection to 34.124.157.94 port 10521
[x] Opening connection to 34.124.157.94 on port 10521
[x] Opening connection to 34.124.157.94 on port 10521: Trying 34.124.157.94
[+] Opening connection to 34.124.157.94 on port 10521: Done
[*] Closed connection to 34.124.157.94 port 10521
[x] Opening connection to 34.124.157.94 on port 10521
[x] Opening connection to 34.124.157.94 on port 10521: Trying 34.124.157.94
[+] Opening connection to 34.124.157.94 on port 10521: Done
[*] Closed connection to 34.124.157.94 port 10521
b'grey{waitttt_I_thought_factorization_is_hard!!?_bSug9kksE3W9SrPL}'


## PLCG (328 points / 32 solves)

The first part was actually done manually, which is to keep retrying until the two lucky numbers are both multiples of 16. We could have written a script with 1/256 chance of success, but by sheer luck we managed it in about 10 tries. Anyway, we obtained:

```
Hello there, this is our lucky numbers
3 80 16 64
Send us your lucky number! 10
Here's your lucky flag: 247692734a39dbf895bc7ce9a38871551c18d744810b1920e6170fa92ccc4b3e2fe6a95ae72292110b78c5065bec72440b317a5ddb296126
Here's your lucky flag: c1a183e98205ddefa55a35925570493198928a1bfccdae69846dd12dec76b05316796efa9c76096c5774ce32766c2421a7c4fe7aa73f191f
Here's your lucky flag: d287ef4d396311abe5403b887c6c5bc85f081cf132c84bf0fe5ae536d815db70c8b233f313a4398606b1b8f6e9e6888fee43d12f405c60d3
Here's your lucky flag: 9e708b90c66feeae44ee33cec21f0eb6e18d32bd39d0923fbbd72dc19b890f87582b70467c1b27ec88811db24f6029d93650078b67a9b050
Here's your lucky flag: 66abaaef5630322ceaf9d64b1ec98843695446f9767e0bd632e661a7b2e9d1dbd0f303913695392084207d35032585ad3f3f59d1b505450a
Here's your lucky flag: 9d6e07c4054e6244654cdb7d0a4b0d0f862d4501e93a2d2e63afe277438bab86a4865399be3be282463d03b344aadaab115706b164b166da
Here's your lucky flag: 12af626ab1d20e221cb4950e5e76d994ec6f4e0645a7ccd414e70ea4740bdf0998d0783e5bc085acba776d7596442509077c971247c16100
Here's your lucky flag: cbef3197bd8b5fca426b6d3f5ad4d95a475d399a802eed6c270bba652df091d0b450ca8bf8ff1c13b3e5cbd85d6fd3ed2353b33eddf4cfa1
Here's your lucky flag: 5b2111328b099a7b9857b886348a8c59b7a1a6004e6837e312507b0b338be1fb077cd7041284d3c817dd2961db5bf21e5a6605c92cd52067
Here's your lucky flag: ae20d773cf0fe43be929f52a359cf9e7afe11648f58230cf53fd691051188b66097c5eaba03e24b234b31eeffdf3e5144d1b18a6386b2b77
```

The trick here is to run the script separately, but print out the keys and keep a frequency table of the bytes.

We find that the byte ends up being one of `[3, 80, 16, 64]` about 61% of the time, and one of `[0, 128, 208]` another 12% of the time. So to make this simple we hope that our key only contains bytes from these 7 possible values.

In [6]:
cts = '''
247692734a39dbf895bc7ce9a38871551c18d744810b1920e6170fa92ccc4b3e2fe6a95ae72292110b78c5065bec72440b317a5ddb296126
c1a183e98205ddefa55a35925570493198928a1bfccdae69846dd12dec76b05316796efa9c76096c5774ce32766c2421a7c4fe7aa73f191f
d287ef4d396311abe5403b887c6c5bc85f081cf132c84bf0fe5ae536d815db70c8b233f313a4398606b1b8f6e9e6888fee43d12f405c60d3
9e708b90c66feeae44ee33cec21f0eb6e18d32bd39d0923fbbd72dc19b890f87582b70467c1b27ec88811db24f6029d93650078b67a9b050
66abaaef5630322ceaf9d64b1ec98843695446f9767e0bd632e661a7b2e9d1dbd0f303913695392084207d35032585ad3f3f59d1b505450a
9d6e07c4054e6244654cdb7d0a4b0d0f862d4501e93a2d2e63afe277438bab86a4865399be3be282463d03b344aadaab115706b164b166da
12af626ab1d20e221cb4950e5e76d994ec6f4e0645a7ccd414e70ea4740bdf0998d0783e5bc085acba776d7596442509077c971247c16100
cbef3197bd8b5fca426b6d3f5ad4d95a475d399a802eed6c270bba652df091d0b450ca8bf8ff1c13b3e5cbd85d6fd3ed2353b33eddf4cfa1
5b2111328b099a7b9857b886348a8c59b7a1a6004e6837e312507b0b338be1fb077cd7041284d3c817dd2961db5bf21e5a6605c92cd52067
ae20d773cf0fe43be929f52a359cf9e7afe11648f58230cf53fd691051188b66097c5eaba03e24b234b31eeffdf3e5144d1b18a6386b2b77
'''

dic = {x[:5]:x[5:] for x in map(bytes.fromhex, cts.split())}
candidate_bytes = [3, 80, 16, 64, 0, 128, 208]
for key in cartesian_product([candidate_bytes] * 6):
    cipher = AES.new(pad(bytes(key), 16), AES.MODE_CTR, nonce=b'\xc1\xc7\xcc\xd1D\xfbI\x10')
    if tail := dic.get(cipher.encrypt(b'grey{')):
        print(b'grey{' + cipher.encrypt(tail))
        break

b'grey{G3T_Rand0m_Byte-is_Still_Bi@s_Oof_7nwh8eQfV5e8eZwC}'


## Encrypt (421 points / 22 solves)

We are given $c = pm + qm^2 + (m+p+q)s$.

The trick here is that $m+p+q$ divides $c+(p+q)(p-pq-q^2)$, so we can use Coppersmith to find the factor $m+p+q$.

In [7]:
n = 60
p = 154086578594169457435595675666643895734811841080572558765373507236578028216591747533849923751469191596377661004029046877904042460778919103625210259448925051403568654035172094553059686620938995150323671690612067502149750334217640430881837803398594614204799922967620005040202036583050736150941842152536365544084
q = 125017463628708786112045898783989519686518641018787292892390877841668306746146702301981172263729102252453507006240973911324033106244068376643435622158128309826549542204034257871545558185841458407254799884829820319949756220781717646450760642511971882897039183232753628829418868493229485015468373205449789812260
c1 = 31118850289098152832161049930974564440792673516199584784484864528279481500612948601526706062621276262711210497739562987491633664814289725255046485262798604510626941827187912034287402128550018798165331343869198539137692903451118993538977788768945912026980846832254010558073806464461172522295653614635829516912620303901074895536704497550933805653512993413784431814034970399353908315083734783641688845887335175756415452320057666293794222522192970247045775053062573130002154959221285571979645935259561842756575513382500001710093979669436220490166791279222321068474420336287079321260681992725702004322840264333436628467610
c2 = 31118850289098152832161049930974564440792673516199584784484864528279481500612948601526706062621276262711210497739562987491633664814289725255046485262798604510626941817672660832127847041917018566902241465270388458210289299587958256824375312920716794521835108724034002277333245660951027397544591256117371462945925063227877052543505162260331377627961855698406102909764518955398788366432268123471930922870790059289526241832032413046933338032005163677585629816264668273416126506175004091486421225900247767247311587061422436593600806854703842740334379936590431991884721985057366825456467986462930236986239275935656810387114

def solve(c):
    x = Zmod(c+(p+q)*(p-p*q-q**2))['x'].gen()
    m = (x+p+q).small_roots(X=2**240, beta=1/3)[0]
    return long_to_bytes(int(m))
print(solve(c1) + solve(c2))

b'grey{shortest_crypto_challenge_in_this_ctf_srfrGRUEShP8FKwn}'


## Fancy (428 points / 21 solves) 🥈

The trick here is that the multiplicative group has order $p^6-1$, and other than that it's just the usual discrete log problem.

In [8]:
p = 2**29 - 33
F = GF(p)['x,y']; x, y = F.gens()
G = F.quo([x**3 - y**2 + 1, y**7 - 11]); x, y = G.gens()
g = 1 + x + y

c = bytes.fromhex('cd519d06bf85ecafdb84111ab63d509e49ffb8cfc78fee4f4cbc3c007a96d2060613f5c0a208325569bf3476d4ea839c10d4667d3dfb5d0d650d79153b')
A = '-210623603*x^2*y^6 + 223917991*x^2*y^5 - 234939507*x*y^6 - 103510738*x^2*y^4 - 255193765*x*y^5 + 245323126*y^6 - 41129482*x^2*y^3 + 3293396*x*y^4 + 265040169*y^5 - 175348566*x^2*y^2 - 8922481*x*y^3 - 76227659*y^4 - 127516194*x^2*y - 97886874*x*y^2 - 207888821*y^3 - 123290485*x^2 + 93703664*x*y - 146824287*y^2 - 229640558*x - 5428142*y - 185137098'
B = '155912203*x^2*y^6 - 50064221*x^2*y^5 + 107681922*x*y^6 - 249464027*x^2*y^4 - 13560651*x*y^5 - 178499062*y^6 + 75225430*x^2*y^3 + 241399622*x*y^4 + 8431316*y^5 - 15433512*x^2*y^2 - 80127041*x*y^3 - 199374666*y^4 + 203619258*x^2*y + 20681482*x*y^2 - 92775952*y^3 - 46663623*x^2 + 171776018*x*y - 164809964*y^2 - 186955302*x + 235677332*y - 173567532'

a = discrete_log(G(A), g, ZZ(p**6-1))
secret = b",".join(map(lambda x : str(x).encode(), (G(B)**a).lift().coefficients()))
print(xor(c, shake_256(secret).digest(len(c))))

b'grey{fancy_group_sometimes_have_small_order_QCuTNsgba9myNdsp}'


## QRSA (454 points / 17 solves) 🩸

Quadratic fields are tricky, so let's just work in $\mathbb{Z}_N[X]/[X^2-41]$ for the most part.

At the end we mod out some common factors so we're left with something small.

In [9]:
N_a = 2613240571441392195964088630982261349682821645613497396226742971850092862049682714123355029612448609254303796690909646594946069650719320421550307082460305103785198772732273571020529003974320397237096691522804712706512030715753640155668659684093067319185265153545236392472134496428382266600090383797614653942221936332929175557303391656241351117808833959918253404012245633586322491783496235954011173498460231177697737092488315432823871012224368640000000
N_b = 406631291381063062708368640624433195177629887128324992156536215422427085251271158548246052765619573442134462500652616281986273622217404519958464200902599497611719198311591180368508835389781999428982410097278062504076636059232055783729252448502542597951710294264137195997893054083787667027206495381119048279226753306334118272352371363733528942151156768581101905518532465160584386180402709606771189313858666352673319676040954150310530906188677120000000
C_a = 2548711194583905242838482900078294859199882484375229964715550469790767416706725411953362845724983002558821710679258499982960453598798074631796750663774845415692650589352513765870894878170769435087683220330986573614974529690187792931316475879984809267941606365493481277785184076320720487644565808909403821593150101568803446075808715002632463329841749179295823686361086890490703942659897558782785569910876849941888829825694107185482012864247559426111336
C_b = 400941158148299866665115436146084555297152646914223433988293961893848206718639579342053294961462797881591789534709492717097892667288044693824228320005182068933966525404665323301134912609777110824069569544060608441451336249895977866445507357131208911196230972379132737483251711155975474018188763433151191428844929401881703566513896999328525340678378000286116960582957867857836600614501387296599091266404311307529322130111164410987643652390537358307965
e = 65537
D = 41

N = N_a**2 - D*N_b**2
F = Zmod(N)['x']
C = F.quo(F.gen()**2-D)((C_a,C_b))

phi = euler_phi(N)
g = gcd(list(C**phi-1)+[N_a,N_b])
d = int(pow(e,-1,phi))
print(b''.join(long_to_bytes(int(x%g)) for x in C**d))

b'grey{x3VkGD3K2SK5s4JW_Lmao_why_do_RSA_in_quadratic_integer}'
